Ce notebook renvoie à la partie 0.2.2 "Aperçu philologique et computationnel des **stemmata codicum** de notre corpus".

In [8]:
%load_ext autoreload
%autoreload 2
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import birth_death_utils as bd
import networkx as nx
from scipy.optimize import fsolve
from tabulate import tabulate
import pandas as pd
import re
from collections import Counter
from scipy.signal.windows import gaussian
from tqdm.notebook import tqdm

plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams['pdf.use14corefonts'] = True 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Partie I : Pré-traitement des *stemmata*


Les données sont les *stemmata* récoltés pour notre corpus de textes patristiques en péninsule Ibérique.Ils sont enregistrés au format `.dot`. Un pré-traitement s'impose: on les charge puis on nettoie les erreurs comme les contaminations, les liens douteux...


In [ ]:
def load_from_OpenStemmata(file):
    '''
    Retourne un arbre orienté (nx.DiGraph) à partir d’un fichier .dot de la base de données.
    '''
    # On charge le graphe depuis le fichier .dot grâce à pydot via networkx
    G = nx.nx_pydot.read_dot(file)
    
    # On supprime les liens de paternité incertaine (arêtes en tirets)
    # Ces liens sont marqués par le style 'dashed' dans le fichier .dot
    edges_pt = nx.get_edge_attributes(G, 'style')
    for edge, pt in edges_pt.items():
        if pt == 'dashed':
            G.remove_edge(*edge)
    
    # On supprime les nœuds isolés (singletons)
    # Ce sont des nœuds sans parents ni enfants, souvent dus à des erreurs ou données inutiles
    singletons = []
    for node in G.nodes():
        if G.in_degree(node) == 0 and G.out_degree(node) == 0:
            singletons.append(node)
    G.remove_nodes_from(singletons)
    
    # On enlève les contaminations intra-stemmatiques
    # Ces contaminations sont des nœuds avec plusieurs parents, ce qui n’est pas possible dans un arbre
    # Pour garder un arbre valide, on conserve un seul parent et on supprime les autres au hasard
    for node in G.nodes():
        in_neighbors = list(G.predecessors(node))
        if G.in_degree(node) > 1:
            contaminations = np.random.choice(in_neighbors, len(in_neighbors) - 1, replace=False)
            for parent in contaminations:
                G.remove_edge(parent, node)
    
    # On identifie les témoins survivants
    # Dans ce contexte, les nœuds sans l’attribut 'color' sont considérés comme témoins vivants
    # On crée un dictionnaire 'living' pour marquer l’état de chaque nœud (True = vivant, False = non)
    colors = nx.get_node_attributes(G, 'color')
    living = {}
    for node in G.nodes():
        if node in colors.keys():
            living[node] = False
        else:
            living[node] = True
    nx.set_node_attributes(G, living, 'state')
    
    # On génère l’arbre stemma final en utilisant une fonction externe
    # Cette fonction supprime notamment les nœuds non attestés qui ne créent pas de branches dans l’arbre
    st = bd.generate_stemma(G)
    
    # On retourne l’arbre nettoyé et prêt à être utilisé
    return st


In [ ]:
#On charge nos arbres
wholeCorpus = {}
for work in os.listdir(f'corpus_stemmata/stemmata_nomodified'):
    print(f'{work}')
    st = load_from_OpenStemmata(f'corpus_stemmata/stemmata_nomodified/{work}/stemma.gv')
    wholeCorpus[f"{work}"] = st

AbellanMartinIglesias_2015_BachiarusDefide
Anglada-Anfruns_2012_Pacianiopera
Arnaud-Lindet_1990_OroseHistoriaadversumpaganos
Bergman_1926_OperaPrudentii
Burgess_1993_ChronicaeIdacii
Daur_1985_OrosiiCommonitorum
Dorfbauer_2015_Dialogus-Quaestionum
Künslte_1905_SyagrusContrahaereticos
Martinez-Diez_1972_MontanusEpistulaeII
Schulgz-Flügel_1994_GregoriusElvireDepithalamioI-II
Schulgz-Flügel_1994_GregoriusElvireDepithalamioIII-V
Zangemeister_1882_OroseLiberapologeticus


## Partie II: Analyse computationnelle des propriétés typologiques des *stemmata*



On commence par charger la liste des *stemmata* à partir du corpus complet. Ensuite, plusieurs indicateurs statistiques sont calculés pour caractériser la structure des arbres :

- **Proportion de nœuds internes de degré 2** : mesure la fréquence des nœuds ayant exactement deux descendants directs.
- **Proportion de nœuds internes de degré 3** : mesure la fréquence des nœuds ayant exactement trois descendants directs.
- **Proportion d’arbres bifides** : indique la part d’arbres à racine bifide.
- **Proportion de filiations directes entre témoins vivants** : évalue la part des liens directs qui relient des témoins survivants dans les arbres.
- **Indice d’asymétrie (imbalance) moyen** : quantifie l’asymétrie moyenne des arbres.

.


In [ ]:
@bd.bootstraped
def prop_degree_2(trees):
    '''
    Retourne la proportion de nœuds internes (non feuilles) de degré deux
    dans une liste d’arbres
    '''
    nb_deg_2 = 0
    nb_internal_nodes = 0
    for g in trees:
        # On compte le nombre total de nœuds internes dans l’arbre
        nb_internal_nodes += len(bd.internal_nodes(g))
        # On vérifie si la racine a un degré sortant égal à 2
        if g.out_degree(bd.root(g)) == 2:
            nb_deg_2 += 1
    # On calcule la proportion de nœuds de degré 2 parmi les nœuds internes
    return nb_deg_2 / nb_internal_nodes


@bd.bootstraped
def prop_degree_3(trees):
    '''
    Retourne la proportion de nœuds internes (non feuilles) de degré trois
    dans une liste d’arbres
    '''
    nb_deg_3 = 0
    nb_internal_nodes = 0
    for g in trees:
        # On compte le nombre total de nœuds internes dans l’arbre
        nb_internal_nodes += len(bd.internal_nodes(g))
        # On vérifie si la racine a un degré sortant égal à 3
        if g.out_degree(bd.root(g)) == 3:
            nb_deg_3 += 1
    # On calcule la proportion de nœuds de degré 3 parmi les nœuds internes
    return nb_deg_3 / nb_internal_nodes


@bd.bootstraped
def bifidity(trees):
    '''
    Retourne la proportion d’arbres dont la racine a un degré sortant de deux
    dans une liste d’arbres
    '''
    nb_bifid = 0
    for g in trees:
        # On vérifie si la racine a deux descendants directs
        if g.out_degree(bd.root(g)) == 2:
            nb_bifid += 1
    # On calcule la proportion d’arbres bifides
    return nb_bifid / len(trees)


@bd.bootstraped
def count_direct_filiation(trees):
    '''
    Retourne la proportion d’arêtes reliant des témoins vivants
    '''
    direct_descents = 0
    nb_edges = 0
    for g in trees:
        for (i, j) in g.edges():
            nb_edges += 1
            # On vérifie si les deux nœuds aux extrémités de l’arête sont vivants
            if g.nodes[i]['state'] and g.nodes[j]['state']:
                direct_descents += 1
    # On calcule la proportion d’arêtes entre témoins vivants
    return direct_descents / nb_edges


@bd.bootstraped
def i3(trees):
    '''
    Calcule la moyenne de l’indice d’asymétrie (imbalance) pour les arbres
    ayant au moins trois feuilles dans une liste d’arbres
    '''
    indices = []
    for g in trees:
        # On conserve uniquement les arbres avec au moins 3 feuilles
        if len(bd.leaves(g)) >= 3:
            indices.append(bd.imbalance_proportion(g))
    # On retourne la moyenne des indices d’asymétrie
    return np.mean(indices)


In [ ]:
# On récupère la liste des stemmata à partir du dictionnaire wholeCorpus
stemmata = list(wholeCorpus.values())

# On calcule différentes mesures statistiques sur les stemmata
deg2 = prop_degree_2(stemmata)          # proportion de nœuds internes de degré 2
deg3 = prop_degree_3(stemmata)          # proportion de nœuds internes de degré 3
bif = bifidity(stemmata)                 # proportion d’arbres avec racine de degré 2 (bifide)
direct = count_direct_filiation(stemmata)  # proportion d’arêtes reliant des témoins vivants
imb = i3(stemmata)                       # moyenne de l’indice d’asymétrie (imbalance)

# On prépare un tableau avec les résultats, leurs valeurs estimées, et leurs bornes inférieure et supérieure
results_iberiancorpus = [
    ['observable', 'valeur estimée', 'borne inférieure', 'borne supérieure'],
    ['prop. stemmata bifides', f'{bif[0]:.2f}', f'{bif[1]:.2f}', f'{bif[2]:.2f}'],
    ['i3 imbalance', f'{imb[0]:.2f}', f'{imb[1]:.2f}', f'{imb[2]:.2f}'],
    ['prop. nœuds internes deg-2', f'{deg2[0]:.2f}', f'{deg2[1]:.2f}', f'{deg2[2]:.2f}'],
    ['prop. nœuds internes deg-3', f'{deg3[0]:.2f}', f'{deg3[1]:.2f}', f'{deg3[2]:.2f}'],
    ['prop. filiation directe entre témoins', f'{direct[0]:.2f}', f'{direct[1]:.2f}', f'{direct[2]:.2f}']
]

# On affiche ce tableau joliment formaté avec la bibliothèque tabulate
print(tabulate(results_iberiancorpus, headers='firstrow', tablefmt='fancy_grid'))


╒══════════════════════════════════╤═══════════════════╤═══════════════╤═══════════════╕
│ observable                       │   estimated value │   lower bound │   upper bound │
╞══════════════════════════════════╪═══════════════════╪═══════════════╪═══════════════╡
│ prop. bifid stemmata             │              0.74 │          0.42 │          1    │
├──────────────────────────────────┼───────────────────┼───────────────┼───────────────┤
│ i3 imbalance                     │              0.16 │          0.02 │          0.36 │
├──────────────────────────────────┼───────────────────┼───────────────┼───────────────┤
│ prop. of deg-2 internal nodes    │              0.08 │          0.04 │          0.21 │
├──────────────────────────────────┼───────────────────┼───────────────┼───────────────┤
│ prop. of deg-3 internal nodes    │              0.02 │          0    │          0.08 │
├──────────────────────────────────┼───────────────────┼───────────────┼───────────────┤
│ prop. of direct wit